In [ ]:
!nvidia-smi


Wed Nov 12 15:46:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4], device='cuda:0')


In [ ]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [ ]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2593, -0.2381, -0.2107,  0.3884,  0.1480, -0.3281,  0.1321, -0.3012,
         -0.0686,  0.1960,  0.0798,  0.4587, -0.4516,  0.4255,  0.1254,  0.0635,
          0.1520,  0.3312,  0.3828, -0.4271],
        [ 0.4247, -0.1323, -0.0925, -0.1126,  0.2209, -0.1501,  0.5302, -0.1136,
         -0.1131,  0.4778,  0.1030,  0.1093, -0.0072, -0.1540,  0.3270,  0.1868,
          0.0263,  0.0352,  0.5887, -0.0230],
        [ 0.5452, -0.1981, -0.1910, -0.2526,  0.5270, -0.4312,  0.5261, -0.2466,
         -0.0019,  0.3713, -0.3943,  0.0812,  0.1067, -0.0086,  0.2960, -0.0837,
         -0.1144, -0.2148,  0.4369, -0.0468]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.2593, 0.0000, 0.0000, 0.3884, 0.1480, 0.0000, 0.1321, 0.0000, 0.0000,
         0.1960, 0.0798, 0.4587, 0.0000, 0.4255, 0.1254, 0.0635, 0.1520, 0.3312,
         0.3828, 0.0000],
        [0.4247, 0.0000, 0.0000, 0.0000, 0.2209, 0.0000, 0.5302, 0.0000, 0.0000,
         0.4778, 0.1030, 0.1093, 0.0000, 0.0000, 0.32

In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [ ]:
print(pred_probab)

tensor([[0.0786, 0.1190, 0.1046, 0.0877, 0.0839, 0.1063, 0.1108, 0.0761, 0.1147,
         0.1184],
        [0.0856, 0.1121, 0.0996, 0.0851, 0.1034, 0.1057, 0.1271, 0.0754, 0.1054,
         0.1006],
        [0.0755, 0.1222, 0.1007, 0.0861, 0.0977, 0.1086, 0.1238, 0.0750, 0.1048,
         0.1056]], grad_fn=<SoftmaxBackward0>)


In [ ]:
print(f"Model structure: {model}\n\n")

for name, param in seq_modules.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: 1.weight | Size: torch.Size([20, 784]) | Values : tensor([[ 0.0066,  0.0137,  0.0189,  ...,  0.0078, -0.0188,  0.0072],
        [-0.0138,  0.0222, -0.0006,  ...,  0.0159,  0.0181,  0.0276]],
       grad_fn=<SliceBackward0>) 

Layer: 1.bias | Size: torch.Size([20]) | Values : tensor([0.0210, 0.0240], grad_fn=<SliceBackward0>) 

Layer: 3.weight | Size: torch.Size([10, 20]) | Values : tensor([[-0.0340, -0.1976, -0.0169,  0.0042,  0.1804, -0.0142,  0.0133,  0.1325,
         -0.2198, -0.0479, -0.1347,  0.0790, -0.2003, -0.1918, -0.2174,  0.1842,
          0.1486, -0.0427,  0.0092,  0.2112],
        [ 0.1108, -0.1393,  0.0219, -0.0770,  0.0376,  0.0309,